<a href="https://colab.research.google.com/github/elamathiJv/llm/blob/main/peft_prefix_tuning_summarization_bart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install Hugging Face Libraries
!pip install "peft==0.2.0"
!pip install "transformers==4.27.1" "datasets==2.9.0" "accelerate==0.17.1" "evaluate==0.4.0" "bitsandbytes==0.37.1" loralib --upgrade --quiet
# install additional dependencies needed for training
!pip install rouge-score tensorboard py7zr

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import os
os.getcwd()

'/content'

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, PrefixTuningConfig, TaskType
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import torch
import os

# os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ["CUDA_VISIBLE_DEVICES"] = "3"

# device = "cuda"
# # device = "cpu"
model_name_or_path = "google/flan-t5-large"
tokenizer_name_or_path = "google/flan-t5-large"

text_column = "document"
label_column = "summary"
# max_length = 512

# lr = 1e-2
# num_epochs = 3
# batch_size = 8

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
tokenizer

T5TokenizerFast(name_or_path='google/flan-t5-large', vocab_size=32100, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'eos_token': '</s>', 'unk_token': '<unk>', 'pad_token': '<pad>', 'additional_special_tokens': ['<extra_id_0>', '<extra_id_1>', '<extra_id_2>', '<extra_id_3>', '<extra_id_4>', '<extra_id_5>', '<extra_id_6>', '<extra_id_7>', '<extra_id_8>', '<extra_id_9>', '<extra_id_10>', '<extra_id_11>', '<extra_id_12>', '<extra_id_13>', '<extra_id_14>', '<extra_id_15>', '<extra_id_16>', '<extra_id_17>', '<extra_id_18>', '<extra_id_19>', '<extra_id_20>', '<extra_id_21>', '<extra_id_22>', '<extra_id_23>', '<extra_id_24>', '<extra_id_25>', '<extra_id_26>', '<extra_id_27>', '<extra_id_28>', '<extra_id_29>', '<extra_id_30>', '<extra_id_31>', '<extra_id_32>', '<extra_id_33>', '<extra_id_34>', '<extra_id_35>', '<extra_id_36>', '<extra_id_37>', '<extra_id_38>', '<extra_id_39>', '<extra_id_40>', '<extra_id_41>', '<extra_id_42>', '<extra_id_43>

In [ ]:
dataset = load_dataset('csv', data_files={'train': 'df_train.csv',
                                              'test': 'df_test.csv'})
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from datasets import concatenate_datasets
import numpy as np
# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["document"], truncation=True), batched=True, remove_columns=["document", "summary"])
input_lenghts = [len(x) for x in tokenized_inputs["input_ids"]]
# take 85 percentile of max length for better utilization
max_source_length = int(np.percentile(input_lenghts, 85))
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets([dataset["train"], dataset["test"]]).map(lambda x: tokenizer(x["summary"], truncation=True), batched=True, remove_columns=["document", "summary"])
target_lenghts = [len(x) for x in tokenized_targets["input_ids"]]
# take 90 percentile of max length for better utilization
max_target_length = int(np.percentile(target_lenghts, 90))
print(f"Max target length: {max_target_length}")

  0%|          | 0/2 [00:00<?, ?ba/s]

Max source length: 512


  0%|          | 0/2 [00:00<?, ?ba/s]

Max target length: 39


In [ ]:
def preprocess_function(sample,padding="max_length"):
    # add prefix to the input for t5
    inputs = ["summarize: " + item for item in sample["document"]]

    # tokenize inputs
    model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(text_target=sample["summary"], max_length=max_target_length, padding=padding, truncation=True)

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_dataset = dataset.map(preprocess_function, batched=True, remove_columns=["document", "summary"])
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path,device_map="auto")

In [ ]:
# peft_config = PrefixTuningConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, num_virtual_tokens=20)
# model = get_peft_model(model, peft_config)
# model.print_trainable_parameters()

In [ ]:
peft_config = PrefixTuningConfig(
    peft_type="PREFIX_TUNING",
    task_type="SEQ_2_SEQ_LM",
    num_virtual_tokens=20,
    num_layers=12
)


In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path,device_map="auto")
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

trainable params: 491520 || all params: 783641600 || trainable%: 0.0627225507170625


In [ ]:
from transformers import DataCollatorForSeq2Seq

# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

output_dir="model_out"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
		auto_find_batch_size=True,
    learning_rate=1e-3, # higher learning rate
    num_train_epochs=5,
    logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    logging_steps=500,
    save_strategy="no",
    report_to="tensorboard",
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
)
model.config.use_cache = False

In [ ]:
trainer.train()

Step,Training Loss
500,4.961000


TrainOutput(global_step=625, training_loss=4.91081240234375, metrics={'train_runtime': 887.3265, 'train_samples_per_second': 5.635, 'train_steps_per_second': 0.704, 'total_flos': 1.152383975424e+16, 'train_loss': 4.91081240234375, 'epoch': 5.0})

In [ ]:
# Save our LoRA model & tokenizer results
peft_model_id="t5_summ_model_01"
trainer.model.save_pretrained(peft_model_id)
tokenizer.save_pretrained(peft_model_id)
# if you want to save the base model to call
# trainer.model.base_model.save_pretrained(peft_model_id)

('t5_summ_model_01/tokenizer_config.json',
 't5_summ_model_01/special_tokens_map.json',
 't5_summ_model_01/tokenizer.json')

In [ ]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

# Load peft config for pre-trained checkpoint etc.
peft_model_id = "t5_summ_model_01"
config = PeftConfig.from_pretrained(peft_model_id)

# load base LLM model and tokenizer
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path,  device_map={"":0})
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id, device_map={"":0})
model.eval()

print("Peft model loaded")

Peft model loaded


In [ ]:
from datasets import load_dataset
from random import randrange

In [ ]:
# Load dataset from the hub and get a sample
sample = dataset['test'][randrange(len(dataset["test"]))]
print(f"document:\n{sample['document']}\n\nactual:\n{sample['summary']}")
input_ids = tokenizer(sample["document"], return_tensors="pt", truncation=True).input_ids.cuda()
# with torch.inference_mode():
outputs = model.generate(input_ids=input_ids, max_new_tokens=60, do_sample=True, top_p=0.9)
# print(f"input sentence: {sample['document']}\n{'---'* 20}")

print(f"summary:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0]}")

document:
The 48-year-old former Arsenal goalkeeper played for the Royals for four years.
He was appointed youth academy director in 2000 and has been director of football since 2003.
A West Brom statement said: "He played a key role in the Championship club twice winning promotion to the Premier League in 2006 and 2012."

actual:
West Brom have appointed Nicky Hammond as technical director, ending his 20-year association with Reading.
summary:
(Analised into the Commonwealth &ampa was announced as "tuation to his own personal fortune and therefore the company's reputation.
